In [1]:
import os
import cv2
import time
import numpy as np


## Working on  Lucas Kanade Optical Flow Estimation
## Working on Video (Not Successful)

In [1]:
import os
import cv2
import time
import numpy as np


cap = cv2.VideoCapture(r'C:\Users\Legion\Downloads\vid.mp4')

feature_params = dict( maxCorners = 100,
                     qualityLevel = 0.1,
                     minDistance = 10,
                     blockSize = 7,
                     useHarrisDetector = True)

lk_params = dict(winSize =(15,15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS| cv2.TERM_CRITERIA_COUNT, 10,0.03))


color = np.random.randint(0, 255, (100, 3))

ret, old_frame = cap.read()
old_frame = cv2.resize(old_frame, (300, 400))
old_grey = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

print(old_frame.shape)
p0 = cv2.goodFeaturesToTrack(old_grey, mask = None, **feature_params)

print("all:",p0)

for p in p0:
    print('just this:')
    print(p[0][0], p[0][1])


mask = np.zeros_like(old_frame)
mask2 = np.zeros_like(old_frame)

#cap.release()
'''
for p in p0:
    x, y = p.ravel()
    mask2 = cv2.circle(mask2, (x,y), 5, (0,255,0),-1)
    
cv2.imshow('frame,' , old_frame)
cv2.imshow('grey', old_grey)
cv2.imshow('w', mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

#'''

p0 = np.array([[[100., 60.]]])
print(type(p0))
while(1):
    
    ret, frame = cap.read()
    
    if ret ==0:
        break
    frame = cv2.resize(frame, (300, 400))
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    p1, status, err = cv2.calcOpticalFlowPyrLK(old_grey, frame_grey, p0 , None, **lk_params)
    print("p1:",p1)
    print("p0", p0)
    print('-------------------')
    
    
    good_new = p1[status == 1]
    good_old = p1[status == 1]
    
    
    print("good new p1:",good_new)
    print("good old p0", good_old)
    print('-------------------')
    
    for idx,(new_pt, old_pt) in enumerate(zip(good_new, good_old)):
        
        a, b = new_pt.ravel()
        c, d = old_pt.ravel()
        
        mask = cv2.line(mask, (a,b), (c,d), color[idx].tolist(),2)
        mask2 = cv2.circle(mask2, (a,b), 5, (0, 255, 0), 1)
        
    img = cv2.add(frame, mask)
    
    cv2.imshow('frame', img)
    cv2.imshow('mask', mask)
    cv2.imshow('mask2', mask2)
    k = cv2.waitKey(1) & 0xff
    
    if k == ord('q'):
        break
    
    old_grey = frame_grey.copy()
    p0 = good_new.reshape(-1,1,2)
    print("shape of p0 before,before p0",p0.shape,p0)
    #any new features found
    new_track_pts = cv2.goodFeaturesToTrack(frame_grey, mask = None, **feature_params)
    print("shape of new pts and new pts",new_track_pts.shape,new_track_pts)
    
    #appending the pts at last
    p0 = np.concatenate((p0, new_track_pts))
    
    print("final shape p0,final p0:",p0.shape,p0)
        
cv2.destroyAllWindows()
cap.release()


(400, 300, 3)
all: [[[ 32.  46.]]

 [[ 38.  34.]]

 [[ 27.  34.]]

 [[ 22.  46.]]]
just this:
32.0 46.0
just this:
38.0 34.0
just this:
27.0 34.0
just this:
22.0 46.0
<class 'numpy.ndarray'>


error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\video\src\lkpyramid.cpp:1229: error: (-215:Assertion failed) (npoints = prevPtsMat.checkVector(2, CV_32F, true)) >= 0 in function 'cv::`anonymous-namespace'::SparsePyrLKOpticalFlowImpl::calc'


In [17]:
import cv2.cuda

In [25]:


import numpy as np
import cv2


help_message = '''
USAGE: opt_flow.py [<video_source>]

Keys:
 1 - toggle HSV flow visualization
 2 - toggle glitch

'''

def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return res

if __name__ == '__main__':
    import sys
    print(help_message)
    
    
    cam = cv2.VideoCapture(r'C:\Users\Legion\Downloads\vid.mp4')
    ret, prev = cam.read()
    prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    show_hsv = False
    show_glitch = False
    cur_glitch = prev.copy()

    while True:
        ret, img = cam.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prevgray, gray, 0.5, 3, 15, 3, 5, int(1.2), 0)
        prevgray = gray

        cv2.imshow('flow', draw_flow(gray, flow))
        if show_hsv:
            cv2.imshow('flow HSV', draw_hsv(flow))
        if show_glitch:
            cur_glitch = warp_flow(cur_glitch, flow)
            cv2.imshow('glitch', cur_glitch)

        ch = 0xFF & cv2.waitKey(5)
        if ch == 27:
            break
        if ch == ord('1'):
            show_hsv = not show_hsv
            print('HSV flow visualization is', ['off', 'on'][show_hsv])
        if ch == ord('2'):
            show_glitch = not show_glitch
            if show_glitch:
                cur_glitch = img.copy()
            print('glitch is', ['off', 'on'][show_glitch])
    cv2.destroyAllWindows()


USAGE: opt_flow.py [<video_source>]

Keys:
 1 - toggle HSV flow visualization
 2 - toggle glitch




TypeError: Required argument 'flags' (pos 10) not found